# 第3章 Chainerの使い方

---

# 3.2 基本オブジェクト

## 3.2.1 Variable

- 変数に入る実際のデータは配列
- Variableの変数の演算結果もVariableの変数になる
- dataという属性で参照できる

In [2]:
import numpy as np
import chainer
from chainer import Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
import chainer.computational_graph as c
from chainer.functions.loss.mean_squared_error import mean_squared_error


x1 = Variable(np.array([1], dtype=np.float32))
x2 = Variable(np.array([2], dtype=np.float32))
x3 = Variable(np.array([3], dtype=np.float32))

z = (x1-2*x2-1)**2+(x2*x3-1)**2+1
z.data

array([ 42.], dtype=float32)

<br>
- 順方向にいったん計算したことになるので、微分値を得るためには、逆向きの計算を行う

In [4]:
z.backward()
x1.grad

In [5]:
x2.grad

In [4]:
x3.grad

array([ 20.], dtype=float32)

#### 微分の式
$$
\frac{\partial z}{\partial x_1}=2(x_1-2x_2-1)\\
\frac{\partial z}{\partial x_2}=-4(x_1-2x_2-1)+2x_3(x_2x_3-1)\\
\frac{\partial z}{\partial x_3}=2x_3(x_2x_3-1)\\
$$
<div style="text-align: center;">
$(x_1,x_2,x_3)=(1,2,3)$を代入すれば<br>　
</div>
$$
\frac{\partial z}{\partial x_1}=2\times(1-2\times2-1)=-8\\
\frac{\partial z}{\partial x_2}=-4\times(1-2\times2-1)+2\times3(2\times3-1)=46\\
\frac{\partial z}{\partial x_3}=2\times3\times(2\times3-1)=20\\
$$

## 3.2.2 Function

In [5]:
x = Variable(np.array([-1], dtype=np.float32))
F.sin(x).data #sin関数

array([-0.84147096], dtype=float32)

In [6]:
F.sigmoid(x).data #sigmoid関数

array([ 0.2689414], dtype=float32)

<br>
- $(\,\cos (x)\,)’ = -\sin(x)$ を確かめてみる

In [7]:
x = Variable(np.array([-0.5], dtype=np.float32))
z = F.cos(x)
z.data

array([ 0.87758255], dtype=float32)

In [8]:
z.backward()
x.grad

array([ 0.47942555], dtype=float32)

In [9]:
(-1)*F.sin(x).data # (cos(x))' = -sin(x)

array([ 0.47942555], dtype=float32)

<br>
- シグモイド関数についても$(\;f'(x)=(1-f(x))\;f(x)\;)$を確かめてみる

In [10]:
x = Variable(np.array([-0.5], dtype=np.float32))
z = F.sigmoid(x)
z.data

array([ 0.37754068], dtype=float32)

In [11]:
z.backward()
x.grad

array([ 0.23500371], dtype=float32)

In [12]:
((1-F.sigmoid(x))*F.sigmoid(x)).data # f'(x)=(1-f(x))*f(x)

array([ 0.23500371], dtype=float32)

<br>
- 変数が多次元である場合は関数の傾きの次元をあらかじめ教えておく必要がある

In [13]:
x = Variable(np.array([-1, 0, 1], dtype=np.float32))
z = F.sin(x)
z.grad = np.ones(3, dtype=np.float32)
z.backward()
x.grad

array([ 0.54030228,  1.        ,  0.54030228], dtype=float32)

<br>
## 3.2.3 links

In [14]:
h = L.Linear(3,4)

- パラメータは$W$と$b$
- 最初に$W$には適当な値が$b$には0が入っている

In [15]:
h.W.data

array([[-0.73528326, -0.42754531, -0.79784918],
       [-0.3103115 ,  0.74695861, -0.09231593],
       [ 0.22093138, -0.93409973,  0.30161089],
       [ 0.65773547, -0.11706574, -0.79936308]], dtype=float32)

In [16]:
h.b.data

array([ 0.,  0.,  0.,  0.], dtype=float32)

<br>
- 入力はバッチ（データの集合）
- 下の例は2つの3次元のベクトルを作って$\;h\;$に与えている

In [17]:
x = Variable(np.array(range(6)).astype(np.float32).reshape(2,3))
x.data

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.]], dtype=float32)

In [18]:
y = h(x)
y.data

array([[-2.02324367,  0.56232679, -0.33087796, -1.71579194],
       [-7.90527678,  1.59532034, -1.56555033, -2.49187183]], dtype=float32)

正しく計算できているのかの確認

In [19]:
w = h.W.data
x0 = x.data
x0.dot(w.T) + h.b.data

array([[-2.02324367,  0.56232679, -0.33087796, -1.71579194],
       [-7.90527678,  1.59532034, -1.56555033, -2.49187183]], dtype=float32)

---

# 3.3 Chainクラス

```python
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1=L.Linear(4, 3),
            l2=L.Linear(3, 3)
        )

    def __call__(self, x, y):
        fv = self.fwd(x, y)
        loss = F.mean_squared_error(fv, y)
        return loss

    def fwd(self, x, y):
        return F.sigmoid(self.l1(x))

```

---

# 3.4 optimizers

```python
model = MyChain() # モデルの生成
optimizer = optimizers.SGD() # 最適化アルゴリズムの選択
optimizer.setup(model) # アルゴリズムにモデルをセット

model.zerograds() # 勾配の初期化
loss = model(x, y) # 順方向に計算して誤差を算出
loss.backward() # 逆方向の計算、勾配の計算
optimizer.update() # パラメータの更新
```

---

# +α　AND・OR・XORの論理演算を学習させてみる

### 参考
[Chainerに入門、And/Or/Xorの実装 ](http://qiita.com/daisukelab/items/6ad3242eeba140023191)

[chainerでニューラルネットを学んでみるよ(chainerでニューラルネット2)](http://hi-king.hatenablog.com/entry/2015/06/27/194630)

## 層が1つの場合

In [8]:
# And/Or/Xor classifier network example
#
# This is re-written version of:
#   http://hi-king.hatenablog.com/entry/2015/06/27/194630
# By following chainer introduction:
#   http://docs.chainer.org/en/stable/tutorial/basic.html

## Chainer cliche
import numpy as np
import chainer
from chainer import Function, Variable, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

# Neural Network

## Network definition
class NN2x2x1dim(Chain):
    def __init__(self):
        super(NN2x2x1dim, self).__init__(
            l = L.Linear(2, 2),
        )
    def __call__(self, x):
        h = self.l(x)
        return h

# Sub routine

## Utility: Summarize current results
def summarize(model, optimizer, inputs, outputs):
    sum_loss, sum_accuracy = 0, 0
    print('model says:')
    for i in range(len(inputs)):
        x  = Variable(inputs[i].reshape(1,2).astype(np.float32))
        t  = Variable(outputs[i].astype(np.int32))
        y = model.predictor(x)
        loss = model(x, t)
        sum_loss += loss.data
        sum_accuracy += model.accuracy.data
        print('  %d & %d = %d (zero:%f one:%f)' % (x.data[0,0], x.data[0,1], np.argmax(y.data), y.data[0,0], y.data[0,1]))
    #mean_loss = sum_loss / len(inputs)
    #mean_accuracy = sum_accuracy / len(inputs)
    #print sum_loss, sum_accuracy, mean_loss, mean_accuracy

## Runs learning loop
def learning_looper(model, optimizer, inputs, outputs, epoch_size):
    augment_size = 100
    for epoch in range(epoch_size):
        print('epoch %d' % epoch)
        for a in range(augment_size):
            for i in range(len(inputs)):
                x = Variable(inputs[i].reshape(1,2).astype(np.float32))
                t = Variable(outputs[i].astype(np.int32))
                optimizer.update(model, x, t)
        summarize(model, optimizer, inputs, outputs)

# Main
## Test data
inputs = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]], dtype=np.float32)
and_outputs = np.array([[0], [0], [0], [1]], dtype=np.int32)
or_outputs = np.array([[0], [1], [1], [1]], dtype=np.int32)
xor_outputs = np.array([[0], [1], [1], [0]], dtype=np.int32)

## AND Test --> will learn successfully
## Model & Optimizer instance
and_model = L.Classifier(NN2x2x1dim())
optimizer = optimizers.SGD()
# quicker) optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(and_model)
print('<<AND: Before learning>>')
summarize(and_model, optimizer, inputs, and_outputs)
print('\n<<AND: After Learning>>')
learning_looper(and_model, optimizer, inputs, and_outputs, epoch_size = 5)

## OR Test --> will learn successfully
## Model & Optimizer instance
or_model = L.Classifier(NN2x2x1dim())
optimizer = optimizers.SGD()
optimizer.setup(or_model)
print('\n---------\n\n<<OR: Before learning>>')
summarize(or_model, optimizer, inputs, or_outputs)
print('\n<<OR: After Learning>>')
learning_looper(or_model, optimizer, inputs, or_outputs, epoch_size = 5)

## XOR Test --> will FAIL, single link is not enough for XOR
## Model & Optimizer instance
xor_model = L.Classifier(NN2x2x1dim())
optimizer = optimizers.SGD()
optimizer.setup(xor_model)
print('\n---------\n\n<<XOR: Before learning>>')
summarize(xor_model, optimizer, inputs, xor_outputs)
print('\n<<XOR: After Learning>>')
learning_looper(xor_model, optimizer, inputs, xor_outputs, epoch_size = 20)


<<AND: Before learning>>
model says:


NameError: name 'y' is not defined

## 層が2つの場合

In [21]:
# Chainer training: And/Or/Xor classifier network example with 2 links.
#
# This is re-written version of:
#   http://hi-king.hatenablog.com/entry/2015/06/27/194630
# By following chainer introduction:
#   http://docs.chainer.org/en/stable/tutorial/basic.html

## Chainer cliche
import numpy as np
import chainer
from chainer import Function, Variable, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F

import chainer.links as L

# Neural Network

## Network definition
class NN2x2_2links(Chain):
    def __init__(self):
        super(NN2x2_2links, self).__init__(
            l1 = L.Linear(2, 2),
            l2 = L.Linear(2, 2),
        )
    def __call__(self, x, y):
        fv = self.forward(x,y)
        loss = F.mean_squared_error(fv,y)
        return loss
    
    def forward(self, x):
        return self.l2(F.sigmoid(self.l1(x)))
        

# Sub routine

## Utility: Summarize current results
"""
def summarize(model, optimizer, inputs, outputs):
    sum_loss, sum_accuracy = 0, 0
    print('model says:')
    for i in range(len(inputs)):
        x  = Variable(inputs[i].reshape(1,2).astype(np.float32))
        t  = Variable(outputs[i].astype(np.int32))
        y = model.predictor(x)
        #loss = model(x, t)
        #sum_loss += loss.data
        #sum_accuracy += model.accuracy.data
        print('  %d & %d = %d (zero:%f one:%f)' % (x.data[0,0], x.data[0,1], np.argmax(y.data), y.data[0,0], y.data[0,1]))
    #mean_loss = sum_loss / len(inputs)
    #mean_accuracy = sum_accuracy / len(inputs)
    #print sum_loss, sum_accuracy, mean_loss, mean_accuracy
"""
## Runs learning loop
def learning_looper(model, optimizer, inputs, outputs, epoch_size):
    augment_size = 100
    for epoch in range(epoch_size):
        print('epoch %d' % epoch)
        for a in range(augment_size):
            for i in range(len(inputs)):
                x = Variable(inputs[i].reshape(1,2).astype(np.float32))
                t = Variable(outputs[i].astype(np.int32))
                #optimizer.update(model, x, t)                
                h = model.forward(x)
                model.zerograds()
                error = F.softmax_cross_entropy(h, t)
                accuracy = F.accuracy(h, t)
                error.backward()
                optimizer.update()
        #summarize(model, optimizer, inputs, outputs)
            print('  %d & %d = %d (zero:%f one:%f)' % (x.data[0,0], x.data[0,1], np.argmax(h.data), h.data[0,0], h.data[0,1]))
        
"""
## Runs XOR_learning loop
def XOR_learning_looper(model, optimizer, inputs, outputs, epoch_size):
    augment_size = 100
    for epoch in range(epoch_size):
        if epoch%10==0:
            print('epoch %d' % epoch)
        for a in range(augment_size):
            for i in range(len(inputs)):
                x = Variable(inputs[i].reshape(1,2).astype(np.float32))
                t = Variable(outputs[i].astype(np.int32))
                optimizer.update(model, x, t)
        if epoch%10==0:
            summarize(model, optimizer, inputs, outputs)
"""        
# Main

## Test data
inputs = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]], dtype=np.float32)
and_outputs = np.array([[0], [0], [0], [1]], dtype=np.int32)
or_outputs = np.array([[0], [1], [1], [1]], dtype=np.int32)
xor_outputs = np.array([[0], [1], [1], [0]], dtype=np.int32)

## AND Test --> will learn successfully
#and_model = L.Classifier(NN2x2_2links())
and_model = NN2x2_2links()
optimizer = optimizers.SGD()
# do it quicker) optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(and_model)
print('<<AND: Before learning>>')
#summarize(and_model, optimizer, inputs, and_outputs)
print('\n<<AND: After Learning>>')
learning_looper(and_model, optimizer, inputs, and_outputs, epoch_size = 21)

## OR Test --> will learn successfully
#or_model = L.Classifier(NN2x2_2links())
or_model = NN2x2_2links()
optimizer = optimizers.SGD()
optimizer.setup(or_model)
print('\n---------\n\n<<OR: Before learning>>')
#summarize(or_model, optimizer, inputs, or_outputs)
print('\n<<OR: After Learning>>')
learning_looper(or_model, optimizer, inputs, or_outputs, epoch_size = 21)

## XOR Test --> will learn successfully
xor_model = L.Classifier(NN2x2_2links())
#optimizer = optimizers.SGD()
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(xor_model)
print('\n---------\n\n<<XOR: Before learning>>')
#summarize(xor_model, optimizer, inputs, xor_outputs)
print('\n<<XOR: After Learning>>')
#XOR_learning_looper(xor_model, optimizer, inputs, xor_outputs, epoch_size = 251)


<<AND: Before learning>>

<<AND: After Learning>>
epoch 0
  1 & 1 = 1 (zero:-0.148278 one:0.521057)
  1 & 1 = 1 (zero:-0.124805 one:0.495775)
  1 & 1 = 1 (zero:-0.101951 one:0.471202)
  1 & 1 = 1 (zero:-0.079705 one:0.447323)
  1 & 1 = 1 (zero:-0.058056 one:0.424125)
  1 & 1 = 1 (zero:-0.036992 one:0.401593)
  1 & 1 = 1 (zero:-0.016501 one:0.379712)
  1 & 1 = 1 (zero:0.003430 one:0.358466)
  1 & 1 = 1 (zero:0.022813 one:0.337841)
  1 & 1 = 1 (zero:0.041663 one:0.317820)
  1 & 1 = 1 (zero:0.059990 one:0.298387)
  1 & 1 = 1 (zero:0.077810 one:0.279528)
  1 & 1 = 1 (zero:0.095134 one:0.261226)
  1 & 1 = 1 (zero:0.111976 one:0.243467)
  1 & 1 = 1 (zero:0.128349 one:0.226234)
  1 & 1 = 1 (zero:0.144266 one:0.209512)
  1 & 1 = 1 (zero:0.159739 one:0.193287)
  1 & 1 = 1 (zero:0.174782 one:0.177543)
  1 & 1 = 0 (zero:0.189406 one:0.162266)
  1 & 1 = 0 (zero:0.203624 one:0.147442)
  1 & 1 = 0 (zero:0.217447 one:0.133057)
  1 & 1 = 0 (zero:0.230889 one:0.119098)
  1 & 1 = 0 (zero:0.243959 one:0.

  1 & 1 = 0 (zero:0.771759 one:-0.316957)
  1 & 1 = 0 (zero:0.771866 one:-0.316406)
  1 & 1 = 0 (zero:0.771971 one:-0.315851)
  1 & 1 = 0 (zero:0.772074 one:-0.315294)
  1 & 1 = 0 (zero:0.772175 one:-0.314735)
  1 & 1 = 0 (zero:0.772274 one:-0.314173)
  1 & 1 = 0 (zero:0.772371 one:-0.313608)
  1 & 1 = 0 (zero:0.772465 one:-0.313041)
  1 & 1 = 0 (zero:0.772558 one:-0.312471)
  1 & 1 = 0 (zero:0.772649 one:-0.311899)
  1 & 1 = 0 (zero:0.772738 one:-0.311325)
  1 & 1 = 0 (zero:0.772825 one:-0.310748)
  1 & 1 = 0 (zero:0.772910 one:-0.310169)
  1 & 1 = 0 (zero:0.772994 one:-0.309588)
  1 & 1 = 0 (zero:0.773076 one:-0.309005)
  1 & 1 = 0 (zero:0.773156 one:-0.308419)
  1 & 1 = 0 (zero:0.773234 one:-0.307831)
  1 & 1 = 0 (zero:0.773310 one:-0.307241)
  1 & 1 = 0 (zero:0.773385 one:-0.306649)
  1 & 1 = 0 (zero:0.773459 one:-0.306055)
  1 & 1 = 0 (zero:0.773530 one:-0.305459)
  1 & 1 = 0 (zero:0.773601 one:-0.304861)
  1 & 1 = 0 (zero:0.773669 one:-0.304261)
  1 & 1 = 0 (zero:0.773736 one:-0.

  1 & 1 = 0 (zero:0.771059 one:-0.160759)
  1 & 1 = 0 (zero:0.770986 one:-0.159967)
  1 & 1 = 0 (zero:0.770912 one:-0.159175)
  1 & 1 = 0 (zero:0.770837 one:-0.158382)
  1 & 1 = 0 (zero:0.770762 one:-0.157589)
  1 & 1 = 0 (zero:0.770686 one:-0.156795)
  1 & 1 = 0 (zero:0.770609 one:-0.156001)
  1 & 1 = 0 (zero:0.770532 one:-0.155205)
  1 & 1 = 0 (zero:0.770454 one:-0.154410)
  1 & 1 = 0 (zero:0.770376 one:-0.153614)
  1 & 1 = 0 (zero:0.770297 one:-0.152817)
  1 & 1 = 0 (zero:0.770217 one:-0.152019)
  1 & 1 = 0 (zero:0.770136 one:-0.151221)
  1 & 1 = 0 (zero:0.770055 one:-0.150422)
  1 & 1 = 0 (zero:0.769973 one:-0.149623)
  1 & 1 = 0 (zero:0.769891 one:-0.148823)
  1 & 1 = 0 (zero:0.769808 one:-0.148023)
  1 & 1 = 0 (zero:0.769724 one:-0.147221)
  1 & 1 = 0 (zero:0.769639 one:-0.146420)
  1 & 1 = 0 (zero:0.769554 one:-0.145618)
  1 & 1 = 0 (zero:0.769468 one:-0.144815)
  1 & 1 = 0 (zero:0.769382 one:-0.144011)
  1 & 1 = 0 (zero:0.769295 one:-0.143207)
  1 & 1 = 0 (zero:0.769207 one:-0.

  1 & 1 = 0 (zero:0.738585 one:0.015707)
  1 & 1 = 0 (zero:0.738335 one:0.016577)
  1 & 1 = 0 (zero:0.738085 one:0.017448)
  1 & 1 = 0 (zero:0.737834 one:0.018319)
  1 & 1 = 0 (zero:0.737582 one:0.019189)
  1 & 1 = 0 (zero:0.737329 one:0.020060)
  1 & 1 = 0 (zero:0.737074 one:0.020931)
  1 & 1 = 0 (zero:0.736819 one:0.021802)
  1 & 1 = 0 (zero:0.736563 one:0.022673)
  1 & 1 = 0 (zero:0.736306 one:0.023544)
  1 & 1 = 0 (zero:0.736048 one:0.024416)
  1 & 1 = 0 (zero:0.735789 one:0.025287)
  1 & 1 = 0 (zero:0.735529 one:0.026158)
  1 & 1 = 0 (zero:0.735268 one:0.027029)
  1 & 1 = 0 (zero:0.735005 one:0.027901)
  1 & 1 = 0 (zero:0.734742 one:0.028772)
  1 & 1 = 0 (zero:0.734478 one:0.029644)
  1 & 1 = 0 (zero:0.734213 one:0.030515)
  1 & 1 = 0 (zero:0.733947 one:0.031387)
  1 & 1 = 0 (zero:0.733680 one:0.032259)
  1 & 1 = 0 (zero:0.733412 one:0.033131)
epoch 7
  1 & 1 = 0 (zero:0.733143 one:0.034002)
  1 & 1 = 0 (zero:0.732873 one:0.034874)
  1 & 1 = 0 (zero:0.732602 one:0.035746)
  1 & 1 

  1 & 1 = 0 (zero:0.663595 one:0.199161)
  1 & 1 = 0 (zero:0.663145 one:0.199999)
  1 & 1 = 0 (zero:0.662694 one:0.200837)
  1 & 1 = 0 (zero:0.662242 one:0.201674)
  1 & 1 = 0 (zero:0.661789 one:0.202511)
  1 & 1 = 0 (zero:0.661336 one:0.203348)
  1 & 1 = 0 (zero:0.660882 one:0.204185)
  1 & 1 = 0 (zero:0.660426 one:0.205021)
epoch 9
  1 & 1 = 0 (zero:0.659971 one:0.205857)
  1 & 1 = 0 (zero:0.659514 one:0.206693)
  1 & 1 = 0 (zero:0.659056 one:0.207528)
  1 & 1 = 0 (zero:0.658598 one:0.208363)
  1 & 1 = 0 (zero:0.658139 one:0.209198)
  1 & 1 = 0 (zero:0.657679 one:0.210032)
  1 & 1 = 0 (zero:0.657218 one:0.210866)
  1 & 1 = 0 (zero:0.656756 one:0.211700)
  1 & 1 = 0 (zero:0.656294 one:0.212534)
  1 & 1 = 0 (zero:0.655831 one:0.213367)
  1 & 1 = 0 (zero:0.655367 one:0.214199)
  1 & 1 = 0 (zero:0.654902 one:0.215032)
  1 & 1 = 0 (zero:0.654436 one:0.215864)
  1 & 1 = 0 (zero:0.653970 one:0.216696)
  1 & 1 = 0 (zero:0.653503 one:0.217528)
  1 & 1 = 0 (zero:0.653035 one:0.218360)
  1 & 1 

...

  1 & 1 = 1 (zero:-1.235469 one:1.132423)
  1 & 1 = 1 (zero:-1.236215 one:1.133288)
  1 & 1 = 1 (zero:-1.236960 one:1.134152)
  1 & 1 = 1 (zero:-1.237706 one:1.135016)
  1 & 1 = 1 (zero:-1.238451 one:1.135880)
  1 & 1 = 1 (zero:-1.239196 one:1.136743)
  1 & 1 = 1 (zero:-1.239941 one:1.137606)
  1 & 1 = 1 (zero:-1.240686 one:1.138468)
  1 & 1 = 1 (zero:-1.241431 one:1.139331)
  1 & 1 = 1 (zero:-1.242176 one:1.140193)
  1 & 1 = 1 (zero:-1.242921 one:1.141054)
  1 & 1 = 1 (zero:-1.243666 one:1.141915)
  1 & 1 = 1 (zero:-1.244410 one:1.142776)
  1 & 1 = 1 (zero:-1.245154 one:1.143637)
  1 & 1 = 1 (zero:-1.245899 one:1.144497)
  1 & 1 = 1 (zero:-1.246643 one:1.145357)
  1 & 1 = 1 (zero:-1.247387 one:1.146217)
  1 & 1 = 1 (zero:-1.248131 one:1.147076)
  1 & 1 = 1 (zero:-1.248874 one:1.147935)
  1 & 1 = 1 (zero:-1.249618 one:1.148793)
  1 & 1 = 1 (zero:-1.250361 one:1.149651)
  1 & 1 = 1 (zero:-1.251105 one:1.150509)
  1 & 1 = 1 (zero:-1.251848 one:1.151367)
  1 & 1 = 1 (zero:-1.252591 one:1.

  1 & 1 = 1 (zero:-1.387039 one:1.304525)
  1 & 1 = 1 (zero:-1.387735 one:1.305301)
  1 & 1 = 1 (zero:-1.388431 one:1.306077)
  1 & 1 = 1 (zero:-1.389127 one:1.306852)
  1 & 1 = 1 (zero:-1.389822 one:1.307627)
  1 & 1 = 1 (zero:-1.390517 one:1.308401)
  1 & 1 = 1 (zero:-1.391212 one:1.309175)
  1 & 1 = 1 (zero:-1.391906 one:1.309949)
  1 & 1 = 1 (zero:-1.392599 one:1.310721)
  1 & 1 = 1 (zero:-1.393293 one:1.311494)
  1 & 1 = 1 (zero:-1.393986 one:1.312265)
  1 & 1 = 1 (zero:-1.394679 one:1.313037)
  1 & 1 = 1 (zero:-1.395371 one:1.313808)
  1 & 1 = 1 (zero:-1.396064 one:1.314578)
  1 & 1 = 1 (zero:-1.396755 one:1.315348)
  1 & 1 = 1 (zero:-1.397447 one:1.316117)
  1 & 1 = 1 (zero:-1.398138 one:1.316886)
  1 & 1 = 1 (zero:-1.398829 one:1.317655)
  1 & 1 = 1 (zero:-1.399519 one:1.318423)
  1 & 1 = 1 (zero:-1.400210 one:1.319190)
  1 & 1 = 1 (zero:-1.400899 one:1.319957)
  1 & 1 = 1 (zero:-1.401589 one:1.320724)
  1 & 1 = 1 (zero:-1.402278 one:1.321490)
  1 & 1 = 1 (zero:-1.402967 one:1.

  1 & 1 = 1 (zero:-1.523453 one:1.454732)
  1 & 1 = 1 (zero:-1.524073 one:1.455407)
  1 & 1 = 1 (zero:-1.524692 one:1.456082)
  1 & 1 = 1 (zero:-1.525312 one:1.456756)
  1 & 1 = 1 (zero:-1.525930 one:1.457429)
  1 & 1 = 1 (zero:-1.526549 one:1.458102)
  1 & 1 = 1 (zero:-1.527166 one:1.458775)
  1 & 1 = 1 (zero:-1.527784 one:1.459447)
  1 & 1 = 1 (zero:-1.528401 one:1.460118)
  1 & 1 = 1 (zero:-1.529018 one:1.460789)
  1 & 1 = 1 (zero:-1.529634 one:1.461460)
  1 & 1 = 1 (zero:-1.530250 one:1.462130)
  1 & 1 = 1 (zero:-1.530866 one:1.462799)
  1 & 1 = 1 (zero:-1.531481 one:1.463469)
  1 & 1 = 1 (zero:-1.532096 one:1.464137)
  1 & 1 = 1 (zero:-1.532710 one:1.464805)
  1 & 1 = 1 (zero:-1.533325 one:1.465473)
  1 & 1 = 1 (zero:-1.533938 one:1.466141)
  1 & 1 = 1 (zero:-1.534552 one:1.466807)
  1 & 1 = 1 (zero:-1.535165 one:1.467474)
  1 & 1 = 1 (zero:-1.535777 one:1.468139)
  1 & 1 = 1 (zero:-1.536389 one:1.468805)
  1 & 1 = 1 (zero:-1.537001 one:1.469470)
  1 & 1 = 1 (zero:-1.537613 one:1.

  1 & 1 = 1 (zero:-1.645377 one:1.586412)
  1 & 1 = 1 (zero:-1.645919 one:1.586993)
  1 & 1 = 1 (zero:-1.646461 one:1.587574)
  1 & 1 = 1 (zero:-1.647002 one:1.588154)
  1 & 1 = 1 (zero:-1.647543 one:1.588734)
  1 & 1 = 1 (zero:-1.648084 one:1.589314)
  1 & 1 = 1 (zero:-1.648624 one:1.589893)
  1 & 1 = 1 (zero:-1.649164 one:1.590471)
  1 & 1 = 1 (zero:-1.649704 one:1.591050)
  1 & 1 = 1 (zero:-1.650243 one:1.591628)
  1 & 1 = 1 (zero:-1.650782 one:1.592205)
  1 & 1 = 1 (zero:-1.651321 one:1.592782)
  1 & 1 = 1 (zero:-1.651859 one:1.593359)
  1 & 1 = 1 (zero:-1.652397 one:1.593935)
  1 & 1 = 1 (zero:-1.652935 one:1.594511)
  1 & 1 = 1 (zero:-1.653472 one:1.595086)
  1 & 1 = 1 (zero:-1.654009 one:1.595661)
  1 & 1 = 1 (zero:-1.654546 one:1.596236)
  1 & 1 = 1 (zero:-1.655082 one:1.596810)
epoch 19
  1 & 1 = 1 (zero:-1.655618 one:1.597384)
  1 & 1 = 1 (zero:-1.656153 one:1.597958)
  1 & 1 = 1 (zero:-1.656688 one:1.598531)
  1 & 1 = 1 (zero:-1.657223 one:1.599103)
  1 & 1 = 1 (zero:-1.6577

  1 & 1 = 1 (zero:-1.750482 one:1.698476)
  1 & 1 = 1 (zero:-1.750957 one:1.698980)
  1 & 1 = 1 (zero:-1.751432 one:1.699483)
  1 & 1 = 1 (zero:-1.751906 one:1.699986)
  1 & 1 = 1 (zero:-1.752380 one:1.700489)
  1 & 1 = 1 (zero:-1.752853 one:1.700991)
  1 & 1 = 1 (zero:-1.753327 one:1.701493)
  1 & 1 = 1 (zero:-1.753800 one:1.701995)
  1 & 1 = 1 (zero:-1.754272 one:1.702496)
  1 & 1 = 1 (zero:-1.754745 one:1.702997)
  1 & 1 = 1 (zero:-1.755217 one:1.703498)
  1 & 1 = 1 (zero:-1.755689 one:1.703998)

---------

<<XOR: Before learning>>

<<XOR: After Learning>>


## こっちでもできます

In [1]:
#!/usr/bin/env python

# 参考元 : http://hi-king.hatenablog.com/entry/2015/06/27/194630

import random
import argparse
import numpy
import chainer
import chainer.optimizers


class SmallClassificationModel(chainer.Chain):
    def __init__(self):
        super(SmallClassificationModel, self).__init__(
            l1 = chainer.links.Linear(2, 2)
            )
    def _forward(self, x):
        h = self.l1(x)
        return h

    def train(self, x_data, y_data):
        x = chainer.Variable(x_data.reshape(1,2).astype(numpy.float32))
        y = chainer.Variable(y_data.astype(numpy.int32))
        h = self._forward(x)

        self.zerograds()
        error = chainer.functions.softmax_cross_entropy(h, y)
        accuracy = chainer.functions.accuracy(h, y)
        error.backward()
        optimizer.update()
        if epoch%100==0:
            print(' %d & %d = %d (zero:%f one:%f)' % (x.data[0,0], x.data[0,1], h.data.argmax(), h.data[0,0], h.data[0,1]))
        #print("error: {}".format(error.data[0]))
        #print("accuracy: {}".format(accuracy.data))

class ClassificationModel(chainer.Chain):
    def __init__(self):
        super(ClassificationModel, self).__init__(
            l1 = chainer.links.Linear(2, 2),
            l2 = chainer.links.Linear(2, 2)
            )
    def _forward(self, x):
        h = self.l2(chainer.functions.sigmoid(self.l1(x)))
        return h

    def train(self, x_data, y_data, epoch):
        x = chainer.Variable(x_data.reshape(1,2).astype(numpy.float32))
        y = chainer.Variable(y_data.astype(numpy.int32))
        h = self._forward(x)

        self.zerograds()
        error = chainer.functions.softmax_cross_entropy(h, y)
        accuracy = chainer.functions.accuracy(h, y)
        error.backward()
        optimizer.update()
        if epoch%100==0:
            print(' %d & %d = %d (zero:%f one:%f)' % (x.data[0,0], x.data[0,1], h.data.argmax(), h.data[0,0], h.data[0,1]))

class RegressionModel(chainer.Chain):
    def __init__(self):
        super(RegressionModel, self).__init__(
            l1 = chainer.links.Linear(2, 2),
            l2 = chainer.links.Linear(2, 1)
            )

    def _forward(self, x):
        h = self.l2(chainer.functions.sigmoid(self.l1(x)))
        return h

    def train(self, x_data, y_data, epoch):
        x = chainer.Variable(x_data.reshape(1,2).astype(numpy.float32))
        y = chainer.Variable(y_data.reshape(1,1).astype(numpy.float32))
        h = self._forward(x)
        self.zerograds()
        error = chainer.functions.mean_squared_error(h, y)
        error.backward()
        optimizer.update()
        if epoch%100==0:
                print('x: {}  h: {})'.format(x.data, h.data))


#model = SmallClassificationModel()     # 層が1つの場合
model = ClassificationModel()           # 層が2つの場合
#model = RegressionModel()              # 重回帰でやった場合

optimizer = chainer.optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(model)

data_xor = [
    [numpy.array([0,0]), numpy.array([0])],
    [numpy.array([0,1]), numpy.array([1])],
    [numpy.array([1,0]), numpy.array([1])],
    [numpy.array([1,1]), numpy.array([0])],
]

data_and = [
    [numpy.array([0,0]), numpy.array([0])],
    [numpy.array([0,1]), numpy.array([0])],
    [numpy.array([1,0]), numpy.array([0])],
    [numpy.array([1,1]), numpy.array([1])],
]

data_or = [
    [numpy.array([0,0]), numpy.array([0])],
    [numpy.array([0,1]), numpy.array([1])],
    [numpy.array([1,0]), numpy.array([1])],
    [numpy.array([1,1]), numpy.array([1])],
]

for epoch in range(1001):
    if epoch%100==0:
        print("epoch: %d" %epoch)
    for invec, outvec in data_xor:
        model.train(invec, outvec, epoch)


epoch: 0
 0 & 0 = 0 (zero:0.878019 one:0.573771)
 0 & 1 = 0 (zero:0.718681 one:0.384091)
 1 & 0 = 0 (zero:0.658738 one:0.436243)
 1 & 1 = 0 (zero:0.514890 one:0.291071)
epoch: 100
 0 & 0 = 0 (zero:0.767173 one:0.674628)
 0 & 1 = 0 (zero:0.545713 one:0.543038)
 1 & 0 = 0 (zero:0.522334 one:0.517389)
 1 & 1 = 1 (zero:0.348787 one:0.435395)
epoch: 200
 0 & 0 = 0 (zero:0.823804 one:0.578874)
 0 & 1 = 1 (zero:0.498525 one:0.543165)
 1 & 0 = 1 (zero:0.433706 one:0.435473)
 1 & 1 = 1 (zero:0.271064 one:0.451696)
epoch: 300
 0 & 0 = 0 (zero:1.005237 one:0.392361)
 0 & 1 = 1 (zero:0.425922 one:0.555449)
 1 & 0 = 1 (zero:0.292928 one:0.404549)
 1 & 1 = 1 (zero:0.180805 one:0.512586)
epoch: 400
 0 & 0 = 0 (zero:1.377135 one:0.117970)
 0 & 1 = 1 (zero:0.335019 one:0.618482)
 1 & 0 = 1 (zero:0.164425 one:0.467239)
 1 & 1 = 1 (zero:0.105640 one:0.616617)
epoch: 500
 0 & 0 = 0 (zero:1.745432 one:-0.165976)
 0 & 1 = 1 (zero:0.287419 one:0.675842)
 1 & 0 = 1 (zero:0.078467 one:0.554091)
 1 & 1 = 1 (zer